#### hypotheses

#### If add maybe work well
- add distance between airports
- add meteo
- add holidays for **from** ans **to**
- mean ticket prices 24h before, avrage before

In [1]:
import numpy as np
import requests
from datetime import datetime
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn import set_config
import pandas as pd
from pandas_profiling import ProfileReport
pd.set_option('display.max_columns', None)

## Data reading

In [2]:
# Old dataframe
df = pd.read_csv('./data/flights_train.csv.bz2')
df_sub = pd.read_csv('./data/flights_Xtest.csv.bz2')

In [3]:
df['flight_date'] = pd.to_datetime(df['flight_date'])
df_sub['flight_date'] = pd.to_datetime(df_sub['flight_date'])

###  UTILS

##### APIs

In [4]:
from joblib import Memory
location = './cachedir'
memory = Memory(location, verbose=0)

headers = {'x-rapidapi-host': "airport-info.p.rapidapi.com",
           'x-rapidapi-key': "670c14cef2mshc0f539c8f496dbcp19f4cejsn7e4b16db61a9"}

def metadata_airport(code):
    url = "https://airport-info.p.rapidapi.com/airport"
    querystring = {"iata" : code, "icao" : code}
    response = requests.request("GET", url, headers=headers, params=querystring)
    # print(response.text)
    response = dict(eval(response.text))
    
    # pd.Series([response["latitude"], response["longitude"], response["country"], response["city"]])
    return pd.Series(response["city"])

metadata_airport = memory.cache(metadata_airport)

metadata_airport('CMN')

0    Nouaceur
dtype: object

In [5]:
# df['from_city'] = df['from'].apply(metadata_airport)
# df['to_city'] = df['to'].apply(metadata_airport)

In [6]:
# df_sub['from_city'] = df_sub['from'].apply(metadata_airport)
# df_sub['to_city'] = df_sub['to'].apply(metadata_airport)

In [7]:
# df[['from_latitude', 'from_longitude', 'from_country']] = df['from'].apply(metadata_airport)
# df[['to_latitude', 'to_longitude', 'to_country']] = df['to'].apply(metadata_airport)

In [8]:
# df_sub[['from_latitude', 'from_longitude', 'from_country']] = df_sub['from'].apply(metadata_airport)
# df_sub[['to_latitude', 'to_longitude', 'to_country']] = df_sub['to'].apply(metadata_airport)

##### Blending Models

In [9]:
def blender(models):
    return np.mean(models, axis=0)

## Data description

In [10]:
df.head()

,flight_date,from,to,avg_weeks,target,std_weeks
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [11]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile

- target variable is symetric 
- there is no hard correlation between target and other dependent variable.

**we need to add meta data, first thing add coordinate for each eaiport, to determine the distance between airports because the price of a flight ticket correlate with the distance, and costumers tend to not let go expensive flight tickets**

the new data is collected thanks to an api that get airports infos

In [12]:
# # New dataframe
df = pd.read_csv('./data/train_with_city.csv')
df_sub = pd.read_csv('./data/test_with_city.csv')

In [13]:
# # New dataframe
# df = pd.read_csv('./data/train_with_coor.csv')
# df_sub = pd.read_csv('./data/test_with_coor.csv')

In [14]:
# df['from_country'].value_counts()

In [15]:
# df['from_country'].value_counts()

- All the flights from and to are domestic ones
- we should drop from_country and to_country because they have one unique value

In [16]:
# df = df.drop(['from_country', 'to_country'], axis=1)

In [17]:
# import geopy.distance
# def calculate_distance(l1, L1, l2, L2):
#     coords_1 = (l1, L1)
#     coords_2 = (l2, L2)
#     return geopy.distance.distance(coords_1, coords_2).km


# df['distance'] = np.vectorize(calculate_distance)(df['from_latitude'], df['from_longitude'], df['to_latitude'], df['to_longitude'])

In [18]:
# df_sub['distance'] = np.vectorize(calculate_distance)(df_sub['from_latitude'], df_sub['from_longitude'], df_sub['to_latitude'], df_sub['to_longitude'])

In [19]:
df

,flight_date,from,to,avg_weeks,target,std_weeks,from_latitude,from_longitude,to_latitude,to_longitude,distance,from_city,to_city
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,41.974163,-87.907320,32.899810,-97.040340,1289.698532,Chicago,DFW Airport
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,36.084000,-115.153740,39.856100,-104.673740,1010.700812,Las Vegas,Denver
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,39.856100,-104.673740,33.941590,-118.408530,1386.808962,Denver,Los Angeles
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,33.640728,-84.427704,41.974163,-87.907320,974.145965,Atlanta,Chicago
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,39.856100,-104.673740,37.621310,-122.378950,1556.562600,Denver,San Francisco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8891,2011-11-24,DEN,PHX,11.045455,9.795363,7.791798,39.856100,-104.673740,33.437267,-112.007790,967.511116,Denver,Phoenix
8892,2012-09-01,ATL,DEN,11.000000,9.429776,7.937254,33.640728,-84.427704,39.856100,-104.673740,1929.754252,Atlanta,Denver
8893,2012-08-10,ORD,EWR,9.476190,12.029971,6.903760,41.974163,-87.907320,40.689533,-74.174460,1157.147451,Chicago,Newark
8894,2012-07-03,ORD,IAH,10.250000,9.381645,7.107261,41.974163,-87.907320,29.990220,-95.336784,1487.504532,Chicago,Houston


In [20]:
print(df['flight_date'].unique().sort)

<built-in method sort of numpy.ndarray object at 0x000001D6DF4C4C10>


In [21]:
# df = df.set_index('flight_date')

#### Pipeline

In [22]:
numeric_preprocessor = Pipeline(steps=[("scaler", StandardScaler())])

In [23]:
categorical_preprocessor = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [24]:
preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ["from", "to"]),
#         ("numerical", numeric_preprocessor, ["avg_weeks", "std_weeks", "distance"])
    ]
)
models = ColumnTransformer(
    [
        ("rf", RandomForestRegressor(n_estimators=1400, min_samples_split=10,
                                     min_samples_leaf=1, max_features='sqrt', 
                                     max_depth=60, bootstrap=True)),
        ("svm", SVR(C=1.0, epsilon=0.2))
    ]
)

In [25]:
# we are going to use pipeline for preprocessing only
pipe = Pipeline(steps=[("preprocessor", preprocessor)]) 

In [26]:
rf_model = RandomForestRegressor(n_estimators=1400, min_samples_split=10,
                                     min_samples_leaf=1, max_features='sqrt', 
                                     max_depth=60, bootstrap=True, n_jobs=-1)

In [27]:
xgb_model = XGBRegressor(n_estimators=3000)

In [28]:
df = pd.read_csv('data/super_data.csv')
df

,from,to,avg_weeks,target,distance,Average Fare ($),Inflation Adjusted Average Fare ($) (Base Year: 2021),2020 Passengers (10% sample)
0,LAS,DEN,14.285714,10.775182,1010.700812,269.623944,312.655018,260905
1,LAS,JFK,14.035714,11.401869,3617.731717,269.623944,312.655018,260905
2,LAS,JFK,17.621622,11.121931,3617.731717,269.623944,312.655018,260905
3,LAS,ORD,12.625000,11.150477,2436.637922,269.623944,312.655018,260905
4,LAS,LAX,16.617647,11.747388,380.412905,269.623944,312.655018,260905
...,...,...,...,...,...,...,...,...
6109,MCO,PHL,14.862069,11.097804,1386.214322,271.096972,320.868370,296484
6110,MCO,PHL,16.900000,10.722479,1386.214322,271.096972,320.868370,296484
6111,MCO,ATL,9.761905,11.038594,649.749056,271.096972,320.868370,296484
6112,MCO,PHL,11.920000,10.488656,1386.214322,271.096972,320.868370,296484


In [37]:
y = df.target
# X = df.drop(['target', 'flight_date', "from_city", "to_city"], axis=1)
X = df.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
X_train_processed = pipe.fit_transform(X_train)
X_test_processed = pipe.transform(X_test)

# X_processed = pipe.fit_transform(X)


In [39]:
# fiting rf
rf_model.fit(X_train_processed, y_train)
# rf_model.fit(X_processed, y)

RandomForestRegressor(max_depth=60, max_features='sqrt', min_samples_split=10,
                      n_estimators=1400, n_jobs=-1)

In [40]:
# fiting xgboost
xgb_model.fit(X_train_processed, y_train)
# xgb_model.fit(X_processed, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
df

,from,to,avg_weeks,target,distance,Average Fare ($),Inflation Adjusted Average Fare ($) (Base Year: 2021),2020 Passengers (10% sample)
0,LAS,DEN,14.285714,10.775182,1010.700812,269.623944,312.655018,260905
1,LAS,JFK,14.035714,11.401869,3617.731717,269.623944,312.655018,260905
2,LAS,JFK,17.621622,11.121931,3617.731717,269.623944,312.655018,260905
3,LAS,ORD,12.625000,11.150477,2436.637922,269.623944,312.655018,260905
4,LAS,LAX,16.617647,11.747388,380.412905,269.623944,312.655018,260905
...,...,...,...,...,...,...,...,...
6109,MCO,PHL,14.862069,11.097804,1386.214322,271.096972,320.868370,296484
6110,MCO,PHL,16.900000,10.722479,1386.214322,271.096972,320.868370,296484
6111,MCO,ATL,9.761905,11.038594,649.749056,271.096972,320.868370,296484
6112,MCO,PHL,11.920000,10.488656,1386.214322,271.096972,320.868370,296484


In [42]:
# # rf prediction
y_rf_pred = rf_model.predict(X_test_processed)

# # xgb prediction
y_xgb_pred = xgb_model.predict(X_test_processed)

In [43]:
y_pred = blender([y_rf_pred, y_xgb_pred])

In [44]:
mse(y_test, y_pred)**0.5

0.6794154365255205

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'regressor__n_estimators': n_estimators,
               'regressor__max_features': max_features,
               'regressor__max_depth': max_depth,
               'regressor__min_samples_split': min_samples_split,
               'regressor__min_samples_leaf': min_samples_leaf,
               'regressor__bootstrap': bootstrap}

grid_search = RandomizedSearchCV(estimator = pipe, param_distributions = param_grid, n_iter = 100,
                          cv = 5, verbose = 2)

In [ ]:
# grid_search.fit(X_train, y_train)

In [ ]:
# grid_search.best_params_

#### Submission

In [ ]:
""" save the new dataframe that containe metadata """
# df.to_csv('data/train_with_city.csv', index=False)
# df_sub.to_csv('data/test_with_city.csv', index=False)

In [ ]:
# pd.read_csv('data/train_with_city.csv')

In [ ]:
df_sub = df_sub.drop(['flight_date', "from_city", "to_city", "to_country", "from_country"], axis=1)


In [ ]:
df

In [ ]:
df_sub

In [ ]:
df_sub = pipe.transform(df_sub)

In [ ]:
y_rf_sub = rf_model.predict(df_sub)
y_xgb_sub = xgb_model.predict(df_sub)

In [ ]:
# df_sub = df_sub.drop(['flight_date', 'from_country', 'to_country'], axis=1)
submition = pd.DataFrame(blender([y_rf_sub, y_xgb_sub]))

In [ ]:
path = 'submition_' + datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.csv'
submition.to_csv(path, index=False, header=False)

In [ ]:
fare_2012 = pd.read_excel('./data/AverageFareAnnual2012.xlsx')
fare_2011 = pd.read_excel('./data/AverageFareAnnual2011.xlsx')
fare_2012['year'] = 2012
fare_2011['year'] = 2011
fare_2012.info()

In [ ]:
for city_df in df[df['year'] == 2012]['from'].unique():
    if city_df not in fare_2012['Airport Code'].unique():
        print(city_df)
        

In [ ]:
fare_2012[fare_2012['City Name'] == 'Chicago']

In [ ]:
df['year'] = pd.DatetimeIndex(df['flight_date']).year
df

In [ ]:
concatenat = pd.concat([fare_2011, fare_2012])
concatenat = concatenat.reset_index()
concatenat

In [ ]:
merged_df = df.merge(concatenat, left_on=['from', 'year'], right_on=['Airport Code', 'year'], how='left')
merged_df

In [ ]:
super_data_exemple = merged_df[['from', 'to', 'avg_weeks', 'target', 'distance', 'Average Fare ($)', 'Inflation Adjusted Average Fare ($) (Base Year: 2021)', '2020 Passengers (10% sample)']]

In [ ]:
# super_data_exemple.to_csv('data/super_data.csv', index=False)

In [ ]:
df.shape